In [2]:
# Ensure you have gurobipy installed:
# !pip install gurobipy
# This is an example for an affine E8-type base consisting of Sp-SO gauge algebras

import gurobipy as gp
from gurobipy import GRB

# Number of A-type, C-type and D-type nodes
n_A = 0
n_C = 5
n_D = 4

# Maximum value for SU(A_i), Sp(C_i) and SO(2D_i)
A_max = 200
C_max = 200
D_max = 200

# Create a new Gurobi model
model = gp.Model("maximize_R")

# Enable non-convex quadratic constraints
model.Params.NonConvex = 2

# Add variables A for A-type, C for C-type nodes and D for D-type nodes
A = model.addVars(n_A, vtype=GRB.INTEGER, name="A", lb=0, ub=A_max)
C = model.addVars(n_C, vtype=GRB.INTEGER, name="C", lb=0, ub=C_max)
D = model.addVars(n_D, vtype=GRB.INTEGER, name="D", lb=0, ub=D_max)

# Define the connections (edges_CD) based on a linear quiver-type tree
# Each edge connects an A-type node to another A-type node
# Each edge connects an A-type node (first) to a C-type node (second)
# Each edge connects an D-type node (first) to a C-type node (second)
# In this example, we define an affine E8-type base consisting of four '-4' and five '-1' tensor multiplets
edges_AA = []
edges_AC = []
edges_CD = [(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3), (3, 3), (3, 4)]

# Define the functions H, V, F, EX, EXF, EXV

# H_AA(i, j) = A_i * A_j for each edge
H_AA = {}
for i, j in edges_AA:
    H_AA[(i, j)] = model.addVar(vtype=GRB.INTEGER, name=f"H_AA_{i}_{j}")
    model.addConstr(H_AA[(i, j)] == A[i] * A[j], name=f"H_AA_Def_{i}_{j}")
    
# H_AC(i, j) = 2A_i * C_j for each edge
H_AC = {}
for i, j in edges_AC:
    H_AC[(i, j)] = model.addVar(vtype=GRB.INTEGER, name=f"H_AC_{i}_{j}")
    model.addConstr(H_AC[(i, j)] == 2 * A[i] * C[j], name=f"H_AC_Def_{i}_{j}")
    
# H_CD(i, j) = 2D_i * C_j for each edge
H_CD = {}
for i, j in edges_CD:
    H_CD[(i, j)] = model.addVar(vtype=GRB.INTEGER, name=f"H_CD_{i}_{j}")
    model.addConstr(H_CD[(i, j)] == 2 * D[i] * C[j], name=f"H_CD_Def_{i}_{j}")


# V(A_i) = A_i^2 - 1
V_A = {}
for i in range(n_A):
    V_A[i] = model.addVar(vtype=GRB.INTEGER, name=f"V_A_{i}")
    model.addConstr(A[i] >= 1, name=f"A_{i}_nonnegative")
    model.addConstr(V_A[i] == A[i] * A[i] - 1, name=f"V_A_def_{j}")

# V(C_j) = C_j(2C_j + 1)
V_C = {}
for j in range(n_C):
    V_C[j] = model.addVar(vtype=GRB.INTEGER, name=f"V_C_{j}")
    model.addConstr(C[j] >= 0, name=f"C_{j}_nonnegative")
    model.addConstr(V_C[j] == C[j] * (2 * C[j] + 1), name=f"V_C_def_{j}")

# V(D_i) = D_i(2D_i - 1)
V_D = {}
for i in range(n_D):
    V_D[i] = model.addVar(vtype=GRB.INTEGER, name=f"V_D_{i}")
    model.addConstr(D[i] >= 4, name=f"D_{i}_nonnegative")
    model.addConstr(V_D[i] == D[i] * (2 * D[i] - 1), name=f"V_F_Def_{i}")

# F_A(j) = number of flavor hypermultiplets charged under A_j
# EXA(A_j) = rank of external gauge algebra
# The sum is over A_i and C_i connected to A_j
F_A = {}
EXA = {}
for j in range(n_A):
    connected_Ai = [i for (i, j_edge) in edges_AA if j_edge == j] + [i for (j_edge, i) in edges_AA if j_edge == j]
    sum_A = gp.quicksum( A[i] for i in connected_Ai)
    connected_Ci = [i for (j_edge, i) in edges_AC if j_edge == j]
    sum_C = gp.quicksum( C[i] for i in connected_Ci)
    F_A[j] = model.addVar(vtype=GRB.INTEGER, name=f"F_A_{j}")
    EXA[j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"EXA_{j}")
    model.addConstr(F_A[j] == (2 * A[j] - sum_A - 2 * sum_C) * A[j], name=f"F_A_def_{j}")
    model.addConstr(F_A[j] >= EXA[j] * A[j], name=f"F_A_{j}_nonnegative")
    
# EXV_A = dim of external gauge algebra
# EXF_A = flavor hypermultiplets for external gauge algebra
EXV_A = {}
EXF_A = {}
for j in range(n_A):
    EXV_A[j] = model.addVar(vtype=GRB.INTEGER, name=f"EXV_A_{j}")
    EXF_A[j] = model.addVar(vtype=GRB.INTEGER, name=f"EXF_A_{j}")
    # EXV_D_j = [2*EXD_j * (2*EXD_j - 1)] / 2
    model.addConstr(EXV_A[j] == EXA[j] * EXA[j], name=f"EXV_A_def_{j}")
    model.addConstr(EXF_A[j] == 2 * EXA[j] * EXA[j] - EXA[j] * A[j], name=f"EXF_A_def_{j}")
    model.addConstr(EXF_A[j] >= 0, name=f"EXF_A_{j}_nonnegative")

# EXD(C_j) = (2C_j + 8) - sum_i (A_i) - sum_i (D_i)
# The sum is over D_i connected to C_j
EXD = {}
for j in range(n_C):
    connected_Ai = [i for (i, j_edge) in edges_AC if j_edge == j]
    sum_A = gp.quicksum( A[i] for i in connected_Ai)
    connected_Di = [i for (i, j_edge) in edges_CD if j_edge == j]
    sum_D = gp.quicksum( D[i] for i in connected_Di)
    EXD[j] = model.addVar(vtype=GRB.INTEGER, name=f"EXD_{j}")
    model.addConstr(EXD[j] == (2 * C[j] + 8) - sum_A - sum_D, name=f"EXD_def_{j}")
    model.addConstr(EXD[j] >= 0, name=f"EXD_{j}_nonnegative")

# F(C_j) = EXD(C_j) * C_j
F_C = {}
for j in range(n_C):
    F_C[j] = model.addVar(vtype=GRB.INTEGER, name=f"F_C_{j}")
    model.addConstr(F_C[j] == 2 * EXD[j] * C[j], name=f"F_C_def_{j}")
    # D(C_j) >= 0
    model.addConstr(F_C[j] >= 0, name=f"F_C_{j}_nonnegative")

# EXV_D(C_j) = [EXD(C_j)/2] * [EXD(C_j)/2 - 1] / 2
EXV_D = {}
for j in range(n_C):
    EXV_D[j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"EXV_D_{j}")
    # EXV_D_j = [2*EXD_j * (2*EXD_j - 1)] / 2
    model.addConstr(2*EXV_D[j] == 2*EXD[j] * (2*EXD[j] - 1), name=f"EXV_D_def_{j}")

# F(D_i) = (2D_i - 8) * 2D_i - sum over j of H(D_i, C_j)
F_D = {}
for i in range(n_D):
    H_CD_sum = gp.quicksum(H_CD[(i, j)] for (i_edge, j) in edges_CD if i_edge == i)
    F_D[i] = model.addVar(vtype=GRB.INTEGER, name=f"F_D_{i}")
    model.addConstr(F_D[i] == (2 * D[i] - 8) * 2 * D[i] - H_CD_sum, name=f"F_D_def_{i}")
    model.addConstr(F_D[i] >= 0, name=f"F_D_{i}_nonnegative")

# Tot constraint
Tot = gp.LinExpr()
Tot += gp.quicksum(H_AA[(i, j)] for (i, j) in edges_AA)
Tot += gp.quicksum(H_AC[(i, j)] for (i, j) in edges_AC)
Tot += gp.quicksum(H_CD[(i, j)] for (i, j) in edges_CD)
Tot += gp.quicksum(F_A[i] for i in range(n_A))
Tot += gp.quicksum(EXF_A[i] for i in range(n_A))
Tot += gp.quicksum(F_C[j] for j in range(n_C))
Tot += gp.quicksum(F_D[i] for i in range(n_D))
Tot -= gp.quicksum(V_A[i] for i in range(n_A))
Tot -= gp.quicksum(V_C[j] for j in range(n_C))
Tot -= gp.quicksum(V_D[i] for i in range(n_D))
Tot += 29 * (n_A + n_C + n_D)
Tot -= gp.quicksum(EXV_A[i] for i in range(n_A))  # Subtract Sum_i EXV_A(A_j)
Tot -= gp.quicksum(EXV_D[j] for j in range(n_C))  # Subtract Sum_j EXV_D(C_j)

# Add the Tot constraint
model.addConstr(Tot <= 273, name="Tot_constraint")

# Objective function: Maximize rank of gauge algebras : R = sum_i C_i + sum_j D_j + sum_k (A_k -1) + external
R = gp.quicksum(A[i] - 1 for i in range(n_A)) + gp.quicksum(C[j] for j in range(n_C)) + gp.quicksum(D[i] for i in range(n_D)) + gp.quicksum(EXA[i] for i in range(n_A)) + gp.quicksum(EXD[j] for j in range(n_C))
model.setObjective(R, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Output the results
if model.status == GRB.OPTIMAL:
    # Print optimal rank of gauge algebras R
    print("\nOptimal value of R:", model.objVal)
    
    A_values = model.getAttr('X', A)
    C_values = model.getAttr('X', C)
    D_values = model.getAttr('X', D)
    EXA_values = model.getAttr('X', EXA)
    EXD_values = model.getAttr('X', EXD)

    # Print A_i for SU(A_i) gauge nodes
    for i in range(n_A):
        print(f"A[{i+1}] = {A_values[i]}")
        # Print the number of flavor hypermultiplets for SU(A_i) gauge nodes
        print(f"EXA[{i+1}] = {EXA_values[i]}")
        
    # Print C_i for Sp(C_i) gauge nodes
    for j in range(n_C):
        print(f"C[{j+1}] = {C_values[j]}")
        # Print the number of flavor hypermultiplets for Sp(C_i) gauge nodes
        print(f"EXD[{j+1}] = {EXD_values[j]}")
    
    # Print D_i for SO(2D_i) gauge nodes
    for i in range(n_D):
        print(f"D[{i+1}] = {D_values[i]}")
else:
    print("No optimal solution found.")

Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 29 rows, 45 columns and 72 nonzeros
Model fingerprint: 0x46b19be9
Model has 31 quadratic constraints
Variable types: 5 continuous, 40 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [2e+00, 4e+00]
  QLMatrix range   [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+02, 2e+02]
  RHS range        [4e+00, 1e+01]
Presolve removed 19 rows and 0 columns
Presolve time: 0.00s
Presolved: 104 rows, 45 columns, 273 nonzeros
Presolved model has 27 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 5 continuous, 40 integer (0 binary)

Root relaxation: objective 2.927070e+03, 57 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current